In [ ]:
 # this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder
FOLDERNAME = 'Scratch/cs224u_scratch'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# this ensures that the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# https://colab.research.google.com/drive/1jocViLorbwWIkTXKwxCOV9HLTaDDgCaw?usp=sharing#scrollTo=iArbRAe781-_

Mounted at /content/drive


In [ ]:
 # this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder
FOLDERNAME = 'CS224u/cs224u/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# this ensures that the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [ ]:
!pip install transformers==4.3.3
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
# import utils
import nltk
import random
# from fuzzywuzzy import process
from multiprocessing import Pool
import json
from transformers import pipeline
import rel_ext

rand = random.SystemRandom()
CURR_DIR = sys.path[-1]

In [ ]:
CORPUS = rel_ext.Corpus(os.path.join(CURR_DIR,'data/rel_ext_data/corpus.tsv.gz'))
#CORPUS = rel_ext.Corpus(os.path.join(CURR_DIR,'data/rel_ext_data/corpus.tsv.gz'))

In [ ]:
# KB = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [ ]:
# DATASET = rel_ext.Dataset(corpus, kb)

In [ ]:
sentences = [
    " ".join([e.left, e.mention_1, e.middle, e.mention_2, e.right])
    for e in CORPUS.examples[:3]
]
print(sentences[0])

# sentence_type_classes = [
#     "Declarative", "Imperative", "Interrogative",
#     "Exclamatory",
# ]

category_classes_long = [
    "History",
    "Economics",
    "Science",
    "Politics",
    "Arts",
    "Sports",
    "Business",
]

def get_classifier():
    return pipeline(
        "zero-shot-classification",
        model="roberta-large-openai-detector",
        # model="distilbert-base-uncased-finetuned-sst-2-english",
        device=0,
    )
  
hypothesis_template = "The the topic of the sentence is {}."
classifier = get_classifier()

def classify(sequence: str, _print=False):
    results = classifier(
        sequence,
        category_classes_long,
        hypothesis_template=hypothesis_template,
    )
    if _print:
        print(results)
    _max = max(results['scores'])
    for category, probability in zip(results["labels"], results['scores']):
        if probability == _max:
            return category

%time print(classify(sentences[2], True))

print("classifying ", len(CORPUS.examples), "examples")

def classify_example(item):
    i, e = item
    text = " ".join([e.left, e.mention_1, e.middle, e.mention_2, e.right])
    return i, e.mention_1,  e.mention_2, classify(text)

2011 History of Town and Charity Background History of the Town Servigliano is a comune ( municipality ) in the Province of Fermo in the Italian region Marche , located about 60 km south of Ancona . As far back as 1914 , with the imminent prospect of Italy entering the war , a large holding camp was built in Servigliano for the


Some weights of the model checkpoint at roberta-large-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


{'sequence': "journeyed to Jaipur , where he studied Panini 's Ashtadhyayi from a Sanskrit scholar . He next journeyed to Ajmer , where he visited the palace of Akbar and the famous Dargah and left for Mount Abu . At Mount Abu , he met Maharaja Ajit Singh of Khetri , who became his ardent devotee and supporter . He was invited to Khetri ,", 'labels': ['Economics', 'Sports', 'Arts', 'Science', 'Politics', 'Business', 'History'], 'scores': [0.16293109953403473, 0.1510338932275772, 0.14647898077964783, 0.1430877149105072, 0.13766346871852875, 0.13160587847232819, 0.1271989643573761]}
Economics
CPU times: user 189 ms, sys: 6.82 ms, total: 196 ms
Wall time: 427 ms
classifying  331696 examples


In [ ]:

with open(
    #"/content/drive/My Drive/Scratch/cs224u_scratch/sentence_classes_more_labels_roberta.txt", 
    "./sentence_topics_roberta.txt", 
        "w+",
) as f:
    for i, e in enumerate(CORPUS.examples):
        i, e1, e2, c = classify_example((i, e))
        f.write(
            f"{i}\t{e1}\t{e2}\t{c}\n"
        )
print("done")